In [2]:
LLAMA = "llama2-70b-4096"
GEMMA = "gemma-7b-it"
MIXTRAL = "mixtral-8x7b-32768"

oracion = "El susurro del viento me transportó a un estado de serenidad profunda que no había experimentado antes."
palabra = "profunda"

In [38]:
from groq import Groq
import json

def get_groq_simplifications(model: str, oracion: str, palabra: str):
    client = Groq(
        api_key="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",
    )

    oracion_ejemplo = "El resplandeciente sol iluminaba el paisaje selvático, creando un efecto casi mágico en la vegetación exuberante que se extendía a lo lejos."
    palabra_ejemplo = "mágico"

    params = {
        "messages": [
            {
                "role": "system",
                "content": f"""
                    Tarea: Sustituir una palabra en una oración sin cambiar el orden de las demás palabras.
                    Respuesta: La respuesta solo incluye los candidatos. Tu respuesta no debe incluir todas las oraciones, solo las palabras por las que se quiere sustituir. La respuesta debe ser en el siguiente formato JSON:
                        {{
                            "candidatos": ["candidato1", "candidato2", "candidato3", "candidato4", "candidato5"],
                        }}

                    Ejemplo de oración original: "{oracion_ejemplo}" Palabra a sustituir: "{palabra_ejemplo}" Respuesta: {{ "candidatos": [ "encantador", "fascinante", "sobrenatural", "misterioso", "maravilloso"] }}
                """
            },
            {
                "role": "user",
                "content": f"""
                    La oración original es: "{oracion}" La palabra a sustituir es "{palabra}".
                    Instrucciones: Proporciona cinco candidatos para sustituir la palabra "{palabra}" en la oración, manteniendo el significado general y la coherencia de la oración. Los candidatos no deben alterar el orden de las demás palabras en la oración.
                """
            }
        ],
        "model": f"{model}",  # Aquí debes reemplazar "model" por el identificador de modelo real que deseas utilizar.
        "max_tokens": 512,
        "temperature": 0.1,
    }


    if model != LLAMA: params["response_format"] = { "type": "json_object" }

    chat_completion = client.chat.completions.create(**params)

    print(f"##### {model} #####")
    print(chat_completion.choices[0].message.content)
    output = json.loads(chat_completion.choices[0].message.content)
    output['candidatos'].append(palabra)

    return output

get_groq_simplifications(GEMMA, oracion, palabra)

##### gemma-7b-it #####
{
  "candidatos": ["encantador", "fascinante", "sobrenatural", "misterioso", "maravilloso"]
}


{'candidatos': ['encantador',
  'fascinante',
  'sobrenatural',
  'misterioso',
  'maravilloso',
  'profunda']}

In [34]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('data/CREA_total.TXT', sep='\t', header=None,
                 names=['Orden', 'Palabra', 'Frec.absoluta', 'Frec.normalizada'],
                 encoding='ISO 8859-1', low_memory=False,
                 dtype={'Orden': str, 'Palabra': str, 'Frec.absoluta': str, 'Frec.normalizada': float})


# Asegurarse de que 'Frec.normalizada' se interprete como numérico
df['Frec.normalizada'] = pd.to_numeric(df['Frec.normalizada'], errors='coerce')

# Ordenar el DataFrame por 'Frec.normalizada' para un mejor gráfico
df_sorted = df.sort_values(by='Frec.normalizada', ascending=True)

# Suponiendo que ya has cargado los datos en el DataFrame df
# Convertir explícitamente la columna 'Palabra' a cadenas de texto
df['Palabra'] = df['Palabra'].astype(str)

# Ordenar el DataFrame por 'Frec.normalizada' para un mejor gráfico
df_sorted = df.sort_values(by='Frec.normalizada', ascending=True)

# Convertir 'Frec.normalizada' a numérico, por si acaso no lo está
df['Frec.normalizada'] = pd.to_numeric(df['Frec.normalizada'], errors='coerce')

In [6]:
df.head()

,Orden,Palabra,Frec.absoluta,Frec.normalizada
0,Orden,Frec.absoluta,Frec.normalizada,NaN
1,1.,de,"9,999,518",65545.55
2,2.,la,"6,277,560",41148.59
3,3.,que,"4,681,839",30688.85
4,4.,el,"4,569,652",29953.48


In [7]:
import pandas as pd
import unicodedata
import json

# Función para remover acentos y caracteres diacríticos
def remover_acentos(input_str):
    if isinstance(input_str, float):  # Verifica si el valor es un float (lo que usualmente indica un NaN en pandas)
        return ""  # Devuelve una cadena vacía si el valor es NaN
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join(c for c in nfkd_form if not unicodedata.combining(c))

def obtener_frecuencias_de_sustitutos(candidatos):
    # Leer el DataFrame asegurándose de que las columnas tienen el tipo correcto
    # Es posible que necesites ajustar los tipos dependiendo de los datos
    df = pd.read_csv('data/CREA_total.TXT', sep='\t', header=None,
                    names=['Orden', 'Palabra', 'Frec.absoluta', 'Frec.normalizada'],
                    encoding='ISO 8859-1', low_memory=False,
                    dtype={'Orden': str, 'Palabra': str, 'Frec.absoluta': str, 'Frec.normalizada': float})

    # Crear una nueva lista con las palabras sin acentos y en minúsculas
    candidatos_minuscula = [remover_acentos(palabra).lower() for palabra in candidatos["candidatos"]]

    # Normalizar la columna 'Palabra' en el DataFrame para que coincida con la lista de candidatos
    df['Palabra'] = df['Palabra'].astype(str).apply(remover_acentos).str.lower().str.strip()

    # Filtrar el DataFrame para obtener solo las filas que contienen las palabras candidatas
    df_filtrado = df[df['Palabra'].isin(candidatos_minuscula)]

    df_unicas_filtradas = df_filtrado.drop_duplicates(subset='Palabra')

    return df_unicas_filtradas

In [9]:
candidatos_mixtral = get_groq_simplifications(MIXTRAL, oracion ,palabra)
df_filtrado_mixtral = obtener_frecuencias_de_sustitutos(candidatos_mixtral)
print(df_filtrado_mixtral)

         Orden       Palabra Frec.absoluta  Frec.normalizada
2160     2160.      profunda         6,817             44.68
7411     7411.    misterioso         1,730             11.33
9546     9546.    fascinante         1,253              8.21
13852   13852.  sobrenatural           769              5.04
15104   15104.     asombroso           682              4.47
18416   18416.    encantador           519              3.40


In [10]:
candidatos_llama = get_groq_simplifications(LLAMA, oracion ,palabra)
df_filtrado_llama = obtener_frecuencias_de_sustitutos(candidatos_llama)
print(df_filtrado_llama)


         Orden       Palabra Frec.absoluta  Frec.normalizada
2160     2160.      profunda         6,817             44.68
6047     6047.   maravilloso         2,214             14.51
7411     7411.    misterioso         1,730             11.33
9546     9546.    fascinante         1,253              8.21
13852   13852.  sobrenatural           769              5.04
18416   18416.    encantador           519              3.40


In [35]:
# ERROR con json.loads()

# candidatos_gemma = get_groq_simplifications(GEMMA, oracion ,palabra)
# df_filtrado_gemma = obtener_frecuencias_de_sustitutos(candidatos_gemma)
# print(df_filtrado_gemma)

In [11]:
def obtener_predicciones(oracion_p: str, palabra_p: str):
    candidatos_mixtral = get_groq_simplifications(MIXTRAL, oracion_p, palabra_p)
    df_filtrado_mixtral = obtener_frecuencias_de_sustitutos(candidatos_mixtral)

    candidatos_llama = get_groq_simplifications(LLAMA, oracion_p, palabra_p)
    df_filtrado_llama = obtener_frecuencias_de_sustitutos(candidatos_llama)

    # combinamos filtrados de los modelos
    df_merged = pd.concat([df_filtrado_mixtral, df_filtrado_llama], ignore_index=True)

    # Ordenamos segun frecuencia absoluta
    df_merged = df_merged.sort_values('Frec.absoluta', ascending=False)

    # Elimino duplicados
    df_merged = df_merged.drop_duplicates(subset='Palabra')

    return df_merged


In [10]:
# Trial data

spanish_input_file = "/home/spoturno/coding/MLSP_Data/Data/Trial/Spanish/multilex_trial_es_ls.tsv" # change this

sentences = []
target_words = []

with open(spanish_input_file, 'r', encoding='utf-8') as file:
    for line in file:
        # Separamos la linea en tabulaciones 
        parts = line.strip().split('\t')
        sentence = parts[0]
        target_word = parts[1]
        sentences.append(sentence)
        target_words.append(target_word)

print(sentences)

['Notifique a su Banco o institución financiera la pérdida o robo de sus tarjetas, chequeras o si sospecha que alguien está utilizando sus cuentas sin su permiso.', 'Una falta de liquidez ocasiona problemas que pueden amenazar la permanencia del negocio en el mercado; problemas como: detención de pago de sueldos, falta de inventario, restricción de crédito de proveedores, falta de capital de trabajo, entre otros.', 'Una falta de liquidez ocasiona problemas que pueden amenazar la permanencia del negocio en el mercado; problemas como: detención de pago de sueldos, falta de inventario, restricción de crédito de proveedores, falta de capital de trabajo, entre otros.', 'Una falta de liquidez ocasiona problemas que pueden amenazar la permanencia del negocio en el mercado; problemas como: detención de pago de sueldos, falta de inventario, restricción de crédito de proveedores, falta de capital de trabajo, entre otros.', 'Cuando se visita a los norteamericanos y se estudian sus leyes, se ve qu

In [18]:
obtener_predicciones("El gato estaba sentado en el vertice de la calle", "vertice")

##### mixtral-8x7b-32768 #####
{
"candidatos": [
"cumbre",
"extremo",
"punta",
"cabecera",
"esquina"
]
}
##### llama2-70b-4096 #####
Sure, here are five candidates to replace the word "vertice" in the sentence "El gato estaba sentado en el vertice de la calle" without changing the order of the other words:

1. Cima: This means "summit" or "peak" in Spanish, which could work in this context to refer to the highest point of the street.
2. Culminación: This means "culmination" or "peak" in Spanish, which could also work to refer to the highest point of the street.
3. Techo: This means "roof" in Spanish, which could work in this context to refer to the highest point of the street, although it might not have the same connotations of "vertice" which implies a pointy peak.
4. Cúspide: This means "summit" or "peak" in Spanish, and could work in a similar way to "vertice" to refer to the highest point of the street.
5. Pico: This means "peak" in Spanish, and could work in a similar way to "vert

JSONDecodeError: Expecting value: line 1 column 1 (char 0)